In [ ]:
# coding: utf-8

# First, start from treating pbc using GROMACS.

# In[ ]:
import sys, os
import numpy as np
#for i in range(20):
#    print i
#    os.chdir('%d'%i)
os.system('echo "1\n" | gmx editconf -f new_push.gro -n index.ndx -o protein.gro')
os.system('echo "1\n" | gmx convert-tpr -s pull.tpr -n index.ndx -o protein.tpr')
os.system('echo "1\n" | gmx trjconv -s protein.tpr -f traj.xtc -o traj_protein.xtc -n index')
os.system('echo "1\n 1\n" | gmx trjconv -s protein.tpr -f traj_protein.xtc -o pbc_cluster.xtc -pbc cluster -n index')
os.system('echo "10\n 1\n" | gmx trjconv -s protein.tpr -f pbc_cluster.xtc -n index.ndx -o cluster_center.xtc -center')
#    os.chdir('../')


# Now, align these traj based upon the reference structure

# In[ ]:

import mdtraj as md
ind = np.array([0,1,2,3,4,5,6,7,8,9,10])
ref = md.load('new_test.pdb')
#for i in range(20):
#    print i
#    os.chdir('%d'%i)
t=md.load('cluster_center.xtc',top='protein.gro')
t_new = t.superpose(ref,atom_indices=ind,ref_atom_indices=ind)
t_new.save_xtc('aligned_cluster_new.xtc')
#    os.chdir('../')

#sys.exit()
# Now, lets treat pbc again to move from original box to a new box (-1.5,1.5)

# In[ ]:
import mdtraj as md
from mdtraj.formats import XTCTrajectoryFile
import numpy as np

#for run in range(20):
#    print run
#    os.chdir('%d'%run)
raw = md.load('aligned_cluster_new.xtc',top='new_test.pdb')
new_xyz = np.zeros((raw.xyz.shape))
n_frame = raw.xyz.shape[0]
n_atom = raw.xyz.shape[1]
n_coor = raw.xyz.shape[2]

for i in range(n_frame):
    for j in range(n_atom):
        for k in range(n_coor):
            if raw.xyz[i,j,k] > 3.00859/2.:
                while raw.xyz[i,j,k] > 3.00859/2.:
                    raw.xyz[i,j,k] = raw.xyz[i,j,k] - 3.00859
                new_xyz[i,j,k] = raw.xyz[i,j,k]
            elif raw.xyz[i,j,k] < -3.00859/2.:
                while raw.xyz[i,j,k] < -3.00859/2.:
                    raw.xyz[i,j,k] = raw.xyz[i,j,k] + 3.00859
                new_xyz[i,j,k] = raw.xyz[i,j,k]
            else:
                new_xyz[i,j,k] = raw.xyz[i,j,k]

with XTCTrajectoryFile('pbc_treat_traj_cluster_new.xtc','w') as f:
    f.write(new_xyz)
np.save('pbc_treat_traj_cluster_new.npy',new_xyz)
#os.chdir('../')


In [ ]:
import sys, os
import numpy as np

for i in range(1):
    print i
    r = []
    org = []
    t = np.load('pbc_treat_traj_cluster_new.npy')
    for j in range(len(t)):
        x,y,z = 0.0,0.0,0.0
        for k in [0,1,2,3,4,5,6,7,9,10]:
            x += t[j][k][0]
            y += t[j][k][1]
            z += t[j][k][2]
        org.append([x/10.,y/10.,z/10.])
        r.append(np.sqrt((t[j][11][0] - x/10.)**2.+(t[j][11][1] - y/10.)**2.+(t[j][11][2] - z/10.)**2.))
    np.save('r.npy',r)
    np.save('org.npy',org)


In [ ]:
import sys, os
import numpy as np

edge=[[9,5,0],[5,2,4],[2,3,10],[3,7,6],[7,9,1],[9,1,0],[5,0,4],[2,4,10],[3,6,10],[7,6,1],[0,1,8],[0,4,8],[4,10,8],[10,6,8],[6,1,8]]
ent = [2,3,7,9,5]
bound = [0,1,2,3,4,5,6,7,9,10]

def comp_cos(v1,v2):
#    cos = np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
    cos = np.dot(v1,v2)/((np.dot(v1,v1)**0.5)*(np.dot(v2,v2)**0.5))
    return cos


for i in range(1):
    print i
    r = []
    org = []
    t = np.load('pbc_treat_traj_cluster_new.npy')
    org = np.load('org.npy')
    cos_total = [[] for frames in range(len(t))]
    for j in range(len(t)):
        v1 = t[j][11] - org[j]
        for ind in edge:
            v2 = (t[j][ind[0]]- org[j]+t[j][ind[1]]- org[j]+t[j][ind[2]]- org[j])/3.
            cos = comp_cos(v1,v2)
            cos_total[j].append(cos)
        v2 = (t[j][ent[0]]- org[j]+t[j][ent[1]]- org[j]+t[j][ent[2]]- org[j]+t[j][ent[3]]- org[j]+t[j][ent[4]]- org[j])/5.
        cos_total[j].append(comp_cos(v1,v2))
#        x,y,z = 0.0,0.0,0.0
#        for l in range(len(bound)):
#            x += t[j][l][0]
#            y += t[j][l][1]
#            z += t[j][l][2]
#        v2 = [x/10.,y/10.,z/10.]
#        cos_total[j].append(comp_cos(v1,v2))            
    np.save('cos.npy',cos_total)


In [ ]:
import sys ,os
import numpy as np
assign = [] # 1 trajs
for i in range(1):
    print i
    cos = np.load('cos.npy')
    r = np.load('r.npy')
    for j in range(len(r)):
        if r[j] <= 0.2:
            assign.append(int(0))
        elif r[j] >= 0.85:
            assign.append(int(17))
        else:
            assign.append(int(np.argmax(cos[j]))+1)  # +1 since state 0 is assigned to bound state
np.save('Assignment.npy',assign)
